In [1]:
from os import preadv
from gevent import config
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from pyspark.sql.streaming import *
from pyspark.sql import SparkSession
import time
import pandas as pd
import config

In [19]:
spark = SparkSession.builder.master("spark://127.0.0.1:7077").appName("mnpl_data_ntbook").set("spark.driver.port", "57411") .getOrCreate()
df = spark.read.parquet("past_data")
df.count()
df2 = df.toPandas()
df2.head()

AttributeError: 'Builder' object has no attribute 'set'

In [33]:
df.loc[df['Symbol'] == 'SJF'].sort_values(by="day", ascending=False)

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,SIGNAL_LINE,PARTITION_DATE
22755,SJF,6.40,6.40,5.96,5.96,1009600.0,6.136374e+09,62,7.2665,9.8056,6.637260,7.822446,-1.185186,-1.291236,20220701
16327,SJF,6.99,6.99,6.39,6.40,787000.0,5.238424e+09,61,7.4835,9.9304,6.760398,7.971442,-1.211044,-1.317748,20220630
16112,SJF,7.08,7.08,6.45,6.75,1701700.0,1.172443e+10,60,7.6935,10.0644,6.825925,8.097157,-1.271232,-1.344424,20220629
15897,SJF,6.23,6.62,6.22,6.62,277000.0,1.807974e+09,59,7.9035,10.2104,6.839730,8.204930,-1.365200,-1.362722,20220628
16757,SJF,6.15,6.25,6.01,6.19,459500.0,2.830447e+09,58,8.1175,10.3800,6.879681,8.331724,-1.452044,-1.362103,20220627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39699,SJF,15.95,16.50,15.95,16.00,1388100.0,2.244100e+10,5,16.4300,16.4300,16.472144,16.537647,-0.065503,-0.015196,20220408
40341,SJF,16.80,16.80,16.20,16.20,1510600.0,2.492898e+10,4,16.5375,16.5375,16.557988,16.580658,-0.022670,-0.002620,20220407
40769,SJF,16.60,17.30,16.50,16.75,2506100.0,4.233229e+10,3,16.6500,16.6500,16.623077,16.611111,0.011966,0.002393,20220406
173,SJF,16.45,16.85,16.40,16.60,1418400.0,2.361533e+10,2,16.6000,16.6000,16.600000,16.600000,0.000000,0.000000,20220405


In [8]:
len(df.Symbol.unique())

866

In [6]:
mapping = df.loc[df['Symbol'] == 'SJF'][['day', 'PARTITION_DATE']]
day_mapping = {}
for ind, row in mapping.iterrows():
    key = row['PARTITION_DATE']
    day = row['day']
    day_mapping[key] = day



In [7]:
for i in range(len(df)):
    row = df.iloc[i]
    day = day_mapping[row['PARTITION_DATE']]
    df.loc[i, 'day'] = day

df.head()

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,PARTITION_DATE
0,AAA,13.10,13.75,12.80,12.80,6651400.0,8.846755e+10,47,12.2775,13.680851,-8.332692,-10.325000,1.992308,20220610
1,AAM,12.20,12.65,12.10,12.40,14100.0,1.737000e+08,47,12.4350,13.540426,-8.580385,-10.558333,1.977949,20220610
2,ABT,41.25,41.25,40.15,41.25,3200.0,1.298400e+08,47,41.2825,40.873404,-28.688846,-35.282407,6.593561,20220610
3,ACB,25.20,25.80,25.15,25.20,2278300.0,5.790451e+10,47,28.2500,30.365957,-20.124231,-24.397222,4.272991,20220610
4,ACL,28.15,28.65,26.65,26.65,605300.0,1.662091e+10,47,26.1225,27.039362,-17.891538,-22.090741,4.199202,20220610


In [24]:
len(df.loc[df['day'] == 62].Symbol.unique())

866

In [8]:
df.sort_values(by='day', ascending=False)

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,PARTITION_DATE
19350,EBS,10.0,10.2,10.00,10.00,1200.0,1.204000e+07,62,10.780,11.206,-7.629615,-9.291667,1.662051,20220701
21139,NLG,35.2,36.5,34.75,36.05,1826800.0,6.480834e+10,62,38.400,43.258,-27.288846,-33.260185,5.971339,20220701
21137,NHC,30.0,30.0,30.00,30.00,100.0,3.000000e+06,62,31.515,31.806,-22.135769,-27.050926,4.915157,20220701
21136,NHA,19.0,20.4,18.60,20.05,144600.0,2.770080e+09,62,22.895,29.670,-16.704808,-20.169907,3.465100,20220701
21135,NET,48.5,48.5,48.50,48.50,1100.0,5.335000e+07,62,48.935,51.538,-34.025385,-41.805556,7.780171,20220701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52132,YBM,11.0,11.2,10.90,10.95,65900.0,7.245250e+08,1,10.950,10.950,NaN,NaN,NaN,20220404
52133,ABR,20.4,20.6,20.05,20.60,600.0,1.224500e+07,1,20.600,20.600,NaN,NaN,NaN,20220404
52134,ACM,3.0,3.1,2.80,2.90,0.0,0.000000e+00,1,2.900,2.900,NaN,NaN,NaN,20220404
52135,ADG,51.8,51.8,51.00,51.60,170500.0,8.778700e+09,1,51.600,51.600,NaN,NaN,NaN,20220404


In [26]:
df['EMA12'] = df['Close']
df['EMA26'] = df['Close']
df

ValueError: cannot insert EMA12, already exists

In [10]:
k12 = 2/13
k26 = 2/27

res = pd.DataFrame(columns=df.columns)
for ticker in config.TICKERS:
    temp_df = df.loc[df["Symbol"] == ticker].copy()
    for i in range(61):
        if i == 0:
            temp_df.loc[temp_df['day'] == 1, 'MACD'] = 0
        if i + 1 not in df[df.Symbol == ticker]['day'].values:
            continue
        prev_ema12 = temp_df.loc[temp_df['day'] == i + 1, 'EMA12'].values[0]
        prev_ema26 = temp_df.loc[temp_df['day'] == i + 1, 'EMA26'].values[0]
        close = temp_df.loc[temp_df['day'] == i + 2, 'Close'].values[0]
        ema12 = close * k12 + prev_ema12 * (1-k12)
        ema26 = close * k26 + prev_ema26 * (1-k26)
        temp_df.loc[temp_df['day'] == i + 2, 'EMA12'] = ema12
        temp_df.loc[temp_df['day'] == i + 2, 'EMA26'] = ema26
        temp_df.loc[temp_df['day'] == i + 2, 'MACD'] = ema12 - ema26
    res = pd.concat([res, temp_df])
    
res.sort_values(by='day', ascending=False)

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,PARTITION_DATE
2214,HMH,16.8,17.4,16.8,17.4,5100.0,85740000.0,62,17.965,19.332,17.519721,18.301295,-0.781574,20220701
21026,CMX,16.8,17.3,16.25,16.95,1297100.0,21693295000.0,62,18.8425,19.04,17.640168,18.46673,-0.826562,20220701
19511,TPC,9.01,9.01,8.51,9.0,2500.0,21543000.0,62,9.3095,9.7134,9.26526,9.402153,-0.136893,20220701
21035,CTCB2201,0.06,0.07,0.05,0.05,611500.0,36500000.0,62,0.186,0.3404,0.135774,0.217877,-0.082102,20220701
20980,APC,18.0,18.5,17.35,18.45,3900.0,69085000.0,62,19.3125,20.534,18.757961,19.56796,-0.809999,20220701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52143,APC,26.4,26.7,26.1,26.6,27100.0,719020000.0,1,26.6,26.6,26.6,26.6,0.0,20220404
52476,FDC,37.0,38.0,36.9,37.0,103800.0,3864440000.0,1,37.0,37.0,37.0,37.0,0.0,20220404
52289,NKG,48.0,48.4,47.25,47.5,4664100.0,222192345000.0,1,47.5,47.5,47.5,47.5,0.0,20220404
52104,UNI,30.9,30.9,29.0,29.0,16300.0,480770000.0,1,29.0,29.0,29.0,29.0,0.0,20220404


In [13]:
res.loc[res["Symbol"] == 'FPT'].sort_values(by='day', ascending=False)

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,PARTITION_DATE
21079,FPT,87.0,88.8,84.8,88.0,1696500.0,146497780000.0,62,94.785,99.872,89.866139,94.49635,-4.630211,20220701
26755,FPT,89.5,89.9,86.2,86.2,1506500.0,132342490000.0,61,96.085,100.352,90.205437,95.016058,-4.810621,20220630
21647,FPT,88.9,90.0,88.2,90.0,1182600.0,105405850000.0,60,97.35,100.868,90.933698,95.721343,-4.787645,20220629
23638,FPT,89.5,90.2,87.2,89.7,2296600.0,204555720000.0,59,98.44,101.378,91.103461,96.17905,-5.075589,20220628
28175,FPT,86.4,87.9,85.9,86.5,1254800.0,109236740000.0,58,99.45,101.884,91.358636,96.697374,-5.338738,20220627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48706,FPT,113.3,113.9,108.5,108.5,4069700.0,450651660000.002991,5,110.62,110.62,110.077228,109.732423,0.344805,20220408
46529,FPT,113.7,116.9,112.1,113.3,3915200.0,450249650000.002014,4,111.15,111.15,110.363996,109.831017,0.53298,20220407
50881,FPT,108.4,114.0,108.4,113.0,4179000.0,465302340000.002991,3,110.433333,110.433333,109.830178,109.553498,0.27668,20220406
51152,FPT,109.1,111.6,109.0,109.0,1863900.0,205138060000.0,2,109.15,109.15,109.253846,109.277778,-0.023932,20220405


In [14]:
schema = (StructType()
            .add(StructField("Symbol", StringType()))
            .add(StructField("Open", StringType()))
            .add(StructField("High", StringType()))
            .add(StructField("Low", StringType()))
            .add(StructField("Close", StringType()))
            .add(StructField("Volume", StringType()))
            .add(StructField("Value", StringType()))
            .add(StructField("day", StringType()))
            .add(StructField("MA20", StringType()))
            .add(StructField("MA50", StringType()))
            .add(StructField("EMA12", StringType()))
            .add(StructField("EMA26", StringType()))
            .add(StructField("MACD", StringType()))
            .add(StructField("PARTITION_DATE", StringType()))    
        )
spark_df = spark.createDataFrame(res, schema=schema)


In [17]:
!export PYSPARK_PYTHON="/usr/local/opt/python@3.8/bin/python3"
!export PYSPARK_DRIVER_PYTHON="/usr/local/opt/python@3.8/bin/python3"

In [3]:
df.head()

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,PARTITION_DATE
0,FUESSV50,22.70,22.90,22.70,22.80,16200.0,3.690190e+08,2,22.650,22.650,22.546154,22.522222,0.023932,20220405
1,FUESSVFL,22.28,22.43,22.24,22.36,26100.0,5.834590e+08,2,22.200,22.200,22.089231,22.063704,0.025527,20220405
2,FUEVFVND,31.11,32.10,30.80,32.05,1720900.0,5.378193e+10,2,32.150,32.150,32.219231,32.235185,-0.015954,20220405
3,FUEVN100,20.46,20.52,20.29,20.35,80700.0,1.643776e+09,2,20.375,20.375,20.392308,20.396296,-0.003989,20220405
4,GAB,196.60,196.60,196.40,196.40,0.0,0.000000e+00,2,196.400,196.400,196.400000,196.400000,0.000000,20220405


In [14]:
df['signal_line'] = df['PARTITION_DATE']
df = df.rename(columns={"PARTITION_DATE":"signal_line", "signal_line" : "PARTITION_DATE"})
df.head()

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,signal_line,PARTITION_DATE
0,FUESSV50,22.70,22.90,22.70,22.80,16200.0,3.690190e+08,2,22.650,22.650,22.546154,22.522222,0.023932,20220405,20220405
1,FUESSVFL,22.28,22.43,22.24,22.36,26100.0,5.834590e+08,2,22.200,22.200,22.089231,22.063704,0.025527,20220405,20220405
2,FUEVFVND,31.11,32.10,30.80,32.05,1720900.0,5.378193e+10,2,32.150,32.150,32.219231,32.235185,-0.015954,20220405,20220405
3,FUEVN100,20.46,20.52,20.29,20.35,80700.0,1.643776e+09,2,20.375,20.375,20.392308,20.396296,-0.003989,20220405,20220405
4,GAB,196.60,196.60,196.40,196.40,0.0,0.000000e+00,2,196.400,196.400,196.400000,196.400000,0.000000,20220405,20220405


In [24]:
df['signal_line'] = df['MACD']
df.head()

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,signal_line,PARTITION_DATE
0,FUESSV50,22.70,22.90,22.70,22.80,16200.0,3.690190e+08,2,22.650,22.650,22.546154,22.522222,0.023932,0.023932,20220405
1,FUESSVFL,22.28,22.43,22.24,22.36,26100.0,5.834590e+08,2,22.200,22.200,22.089231,22.063704,0.025527,0.025527,20220405
2,FUEVFVND,31.11,32.10,30.80,32.05,1720900.0,5.378193e+10,2,32.150,32.150,32.219231,32.235185,-0.015954,-0.015954,20220405
3,FUEVN100,20.46,20.52,20.29,20.35,80700.0,1.643776e+09,2,20.375,20.375,20.392308,20.396296,-0.003989,-0.003989,20220405
4,GAB,196.60,196.60,196.40,196.40,0.0,0.000000e+00,2,196.400,196.400,196.400000,196.400000,0.000000,0.000000,20220405


In [25]:
k9 = 2/10
res = pd.DataFrame(columns=df.columns)
for ticker in config.TICKERS:
    temp_df = df.loc[df["Symbol"] == ticker].copy()
    for i in range(61):
        if i + 1 not in df[df.Symbol == ticker]['day'].values:
            continue
        prev_ema9 = temp_df.loc[temp_df['day'] == i + 1, 'signal_line'].values[0]
        close = temp_df.loc[temp_df['day'] == i + 2, 'MACD'].values[0]
        ema9 = close * k9 + prev_ema9 * (1-k9)
        temp_df.loc[temp_df['day'] == i + 2, 'signal_line'] = ema9
    res = pd.concat([res, temp_df])
    

In [27]:
res.loc[res.Symbol == 'FPT'].sort_values(by='day', ascending=False)

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,signal_line,PARTITION_DATE
16092,FPT,87.0,88.8,84.8,88.0,1696500.0,146497780000.0,62,94.785,99.872,89.866139,94.49635,-4.630211,-4.359203,20220701
13548,FPT,89.5,89.9,86.2,86.2,1506500.0,132342490000.0,61,96.085,100.352,90.205437,95.016058,-4.810621,-4.291451,20220630
14010,FPT,88.9,90.0,88.2,90.0,1182600.0,105405850000.0,60,97.35,100.868,90.933698,95.721343,-4.787645,-4.161658,20220629
15399,FPT,89.5,90.2,87.2,89.7,2296600.0,204555720000.0,59,98.44,101.378,91.103461,96.17905,-5.075589,-4.005161,20220628
14242,FPT,86.4,87.9,85.9,86.5,1254800.0,109236740000.0,58,99.45,101.884,91.358636,96.697374,-5.338738,-3.737555,20220627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7293,FPT,113.3,113.9,108.5,108.5,4069700.0,450651660000.002991,5,110.62,110.62,110.077228,109.732423,0.344805,0.187202,20220408
8484,FPT,113.7,116.9,112.1,113.3,3915200.0,450249650000.002014,4,111.15,111.15,110.363996,109.831017,0.53298,0.147801,20220407
2004,FPT,108.4,114.0,108.4,113.0,4179000.0,465302340000.002991,3,110.433333,110.433333,109.830178,109.553498,0.27668,0.051507,20220406
596,FPT,109.1,111.6,109.0,109.0,1863900.0,205138060000.0,2,109.15,109.15,109.253846,109.277778,-0.023932,-0.004786,20220405


In [34]:
# df["Up_Change"] = df["MA20"]
# df["Down_Change"] = df["MA20"]
df.insert(loc=14, column='Up_Change', value=df['Close'])
df.insert(loc=15, column='Down_Change', value=df['Close'])
df

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,SIGNAL_LINE,Up_Change,Down_Change,PARTITION_DATE
0,FUESSVFL,22.28,22.43,22.24,22.36,26100.0,5.834590e+08,2,22.20000,22.20000,22.089231,22.063704,0.025527,0.005105,22.36,22.36,20220405
1,FUEVFVND,31.11,32.10,30.80,32.05,1720900.0,5.378193e+10,2,32.15000,32.15000,32.219231,32.235185,-0.015954,-0.003191,32.05,32.05,20220405
2,FUEVN100,20.46,20.52,20.29,20.35,80700.0,1.643776e+09,2,20.37500,20.37500,20.392308,20.396296,-0.003989,-0.000798,20.35,20.35,20220405
3,GAB,196.60,196.60,196.40,196.40,0.0,0.000000e+00,2,196.40000,196.40000,196.400000,196.400000,0.000000,0.000000,196.40,196.40,20220405
4,GAS,112.70,114.80,111.00,114.30,727800.0,8.262988e+10,2,113.35000,113.35000,112.692308,112.540741,0.151567,0.030313,114.30,114.30,20220405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52666,FUCVREIT,10.25,10.30,10.00,10.00,2300.0,2.311000e+07,8,10.16875,10.16875,10.120627,10.076888,0.043738,0.035906,10.00,10.00,20220414
52667,FUEIP100,11.05,11.06,10.94,10.97,53900.0,5.950800e+08,8,11.09500,11.09500,11.105322,11.174273,-0.068950,-0.033792,10.97,10.97,20220414
52668,FUEKIV30,10.00,10.04,9.95,9.95,56300.0,5.630390e+08,8,10.01875,10.01875,10.000125,9.990919,0.009206,0.010824,9.95,9.95,20220414
52669,FUEMAV30,17.95,17.95,17.70,17.86,14400.0,2.571170e+08,8,17.94125,17.94125,17.941884,17.964140,-0.022256,-0.009086,17.86,17.86,20220414


In [14]:
res = pd.DataFrame(columns=df.columns)
for ticker in config.TICKERS:
    temp_df = df.loc[df["Symbol"] == ticker].copy()
    for i in range(62):
        if i == 0:
            temp_df.loc[temp_df['day'] == 1, "Down_Change"] = 0
            temp_df.loc[temp_df['day'] == 1, "Up_Change"] = 0
        if i not in df[df.Symbol == ticker]['day'].values:
            continue
        prev_close = temp_df.loc[temp_df['day'] == i, "Close"].values[0]
        cur_close = temp_df.loc[temp_df['day'] == i + 1, "Close"].values[0]
        if prev_close < cur_close:
            temp_df.loc[temp_df['day'] == i + 1, "Down_Change"] = 0
            temp_df.loc[temp_df['day'] == i + 1, "Up_Change"] = cur_close - prev_close
        else:
            temp_df.loc[temp_df['day'] == i + 1, "Down_Change"] = prev_close - cur_close
            temp_df.loc[temp_df['day'] == i + 1, "Up_Change"] =  0
    res = pd.concat([res, temp_df])
    
res.loc[res.Symbol=="SJF"].sort_values(by="day")

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,SIGNAL_LINE,PARTITION_DATE,Up_Change,Down_Change
1795,SJF,17.0,17.3,16.6,16.6,2038100.0,34421485000.0,1,16.6,16.6,16.6,16.6,0.0,0.0,20220404,0.0,0.0
173,SJF,16.45,16.85,16.4,16.6,1418400.0,23615325000.000099,2,16.6,16.6,16.6,16.6,0.0,0.0,20220405,0.0,0.0
40769,SJF,16.6,17.3,16.5,16.75,2506100.0,42332290000.000099,3,16.65,16.65,16.623077,16.611111,0.011966,0.002393,20220406,0.15,0.0
40341,SJF,16.8,16.8,16.2,16.2,1510600.0,24928985000.0,4,16.5375,16.5375,16.557988,16.580658,-0.02267,-0.00262,20220407,0.0,0.55
39699,SJF,15.95,16.5,15.95,16.0,1388100.0,22441000000.0,5,16.43,16.43,16.472144,16.537647,-0.065503,-0.015196,20220408,0.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16757,SJF,6.15,6.25,6.01,6.19,459500.0,2830446999.99999,58,8.1175,10.38,6.879681,8.331724,-1.452044,-1.362103,20220627,0.09,0.0
15897,SJF,6.23,6.62,6.22,6.62,277000.0,1807974000.0,59,7.9035,10.2104,6.83973,8.20493,-1.3652,-1.362722,20220628,0.43,0.0
16112,SJF,7.08,7.08,6.45,6.75,1701700.0,11724429000.0,60,7.6935,10.0644,6.825925,8.097157,-1.271232,-1.344424,20220629,0.13,0.0
16327,SJF,6.99,6.99,6.39,6.4,787000.0,5238424000.0,61,7.4835,9.9304,6.760398,7.971442,-1.211044,-1.317748,20220630,0.0,0.35


In [16]:
df = res

In [35]:
df.insert(loc=16, column='RSI', value=df['Up_Change'])
df

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,SIGNAL_LINE,Up_Change,RSI,Down_Change,PARTITION_DATE
0,FUESSVFL,22.28,22.43,22.24,22.36,26100.0,5.834590e+08,2,22.20000,22.20000,22.089231,22.063704,0.025527,0.005105,22.36,22.36,22.36,20220405
1,FUEVFVND,31.11,32.10,30.80,32.05,1720900.0,5.378193e+10,2,32.15000,32.15000,32.219231,32.235185,-0.015954,-0.003191,32.05,32.05,32.05,20220405
2,FUEVN100,20.46,20.52,20.29,20.35,80700.0,1.643776e+09,2,20.37500,20.37500,20.392308,20.396296,-0.003989,-0.000798,20.35,20.35,20.35,20220405
3,GAB,196.60,196.60,196.40,196.40,0.0,0.000000e+00,2,196.40000,196.40000,196.400000,196.400000,0.000000,0.000000,196.40,196.40,196.40,20220405
4,GAS,112.70,114.80,111.00,114.30,727800.0,8.262988e+10,2,113.35000,113.35000,112.692308,112.540741,0.151567,0.030313,114.30,114.30,114.30,20220405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52666,FUCVREIT,10.25,10.30,10.00,10.00,2300.0,2.311000e+07,8,10.16875,10.16875,10.120627,10.076888,0.043738,0.035906,10.00,10.00,10.00,20220414
52667,FUEIP100,11.05,11.06,10.94,10.97,53900.0,5.950800e+08,8,11.09500,11.09500,11.105322,11.174273,-0.068950,-0.033792,10.97,10.97,10.97,20220414
52668,FUEKIV30,10.00,10.04,9.95,9.95,56300.0,5.630390e+08,8,10.01875,10.01875,10.000125,9.990919,0.009206,0.010824,9.95,9.95,9.95,20220414
52669,FUEMAV30,17.95,17.95,17.70,17.86,14400.0,2.571170e+08,8,17.94125,17.94125,17.941884,17.964140,-0.022256,-0.009086,17.86,17.86,17.86,20220414


In [19]:
res = pd.DataFrame(columns=df.columns)
for ticker in config.TICKERS:
    temp_df = df.loc[df["Symbol"] == ticker].copy()
    temp_df = temp_df.sort_values(by="day")
    temp_df["RSI"] = 100 - 100 / (1 + temp_df["Up_Change"].rolling(14).mean() / (temp_df["Down_Change"].rolling(14).mean() + 1e-6))
    res = pd.concat([res, temp_df])

res.sort_values(by="day").tail(10)

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,SIGNAL_LINE,PARTITION_DATE,Up_Change,Down_Change,RSI
22744,SFG,13.15,13.95,13.15,13.5,1900.0,25405000.0,62,14.46,15.51,13.925763,14.594352,-0.668589,-0.675379,20220701,0.0,0.6,31.111014
52206,TJC,15.0,15.1,14.7,14.7,0.0,0.0,62,16.33,16.554,15.755655,16.175077,-0.419422,-0.204448,20220701,0.0,0.0,0.0
22745,SFI,61.9,61.9,58.6,58.6,2100.0,128890000.0,62,63.0,69.218,61.765554,64.249217,-2.483662,-2.720025,20220701,0.0,3.9,32.820489
15413,CVIC2206,0.04,0.51,0.04,0.47,59200.0,29992000.0,62,0.656,0.766471,0.566738,0.696189,-0.129451,NaN,20220701,0.0,0.0,19.564622
22746,SGN,66.9,67.5,65.3,67.5,4900.0,327260000.0,62,67.75,68.508,67.569128,67.966688,-0.397559,-0.549835,20220701,0.0,0.5,43.749936
22747,SGR,17.5,17.5,16.55,17.3,50600.0,863275000.0,62,17.215,19.908,17.082252,18.029617,-0.947364,-1.29308,20220701,0.0,0.2,48.175084
52205,TIG,12.2,12.8,11.8,12.5,280800.0,3410390000.0,62,14.03,15.838,13.144771,14.331957,-1.187186,-1.208236,20220701,0.0,0.1,29.508129
22748,SGT,25.1,25.6,24.4,25.6,33000.0,828755000.0,62,26.45,28.005,25.906821,26.767466,-0.860645,-0.96204,20220701,0.0,0.4,30.833261
15414,CVJC2201,0.15,0.15,0.1,0.12,356000.0,43198000.0,62,0.1975,0.3408,0.174778,0.233998,-0.059221,NaN,20220701,0.0,0.03,37.929203
52274,X20,9.1,9.1,9.1,9.1,0.0,0.0,62,9.145,9.852,8.96089,9.255723,-0.294833,-0.368841,20220701,0.0,0.0,48.717774


In [39]:
spark.stop()

In [1]:
import json 
import pandas as pd

df1 = pd.DataFrame(columns=['Symbol', 'TradingDate', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
with open("DATN/src/ingestion/producer/test_data.json", 'r') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        row = [data['Symbol'], data['TradingDate'], data['Time'], data['Open'], \
                data['High'], data['Low'], data['Close'], data['Volume']]
        df1.loc[len(df1)] = row
df1 = df1.sort_values('Time')
df1

FileNotFoundError: [Errno 2] No such file or directory: 'DATN/src/ingestion/producer/test_data.json'